# LinkedIn Learning Courses Analysis

This notebook analyses LinkedIn Courses taken by CaseWare employees. Created as part of hackathon 2020 by Oliver Phan,
Ekat Zaytseva, Irina Balzamova, and Shweta Chandratre.

In [ ]:
# tags: configuration
from cwi_notebook.configuration import notebook, inputs, outputs
from cwi_taxonomy.v0.tags import *
notebook.id="linkedin_learning_courses_analysis"
notebook.version="1.0.0"
notebook.autorun=False

outputs.transaction_scores.columns = [AMOUNT, POSTING_DATE, ACCOUNT_ID, ACCOUNT_MAIN_DESCRIPTION]

In [ ]:
import pandas as pd

AUTH_ENDPOINT = 'http://localhost:3000/auth'
ORGANIZATIONS_ENDPOINT = 'http://localhost:3000/organizations/{0}'
ACCOUNTS_ENDPOINT = 'http://localhost:3000/organizations/{0}/accounts/{1}'
TRANSACTIONS_ENDPOINT = 'http://localhost:3000/organizations/{0}/accounts/{1}/transactions{2}'

# TODO: Setup Linkedin endpoints to agree with these ids for filtering
LINKEDIN_ORGID = 123
COURSE_ID = 123
COURSE_VIEW_ID = 123


# TODO: Setup Linkedin binding with agreed upon user/password & returned access token format
def authenticate_against_p19k():
    provider_bindings_token = 
        requests.post(AUTH_ENDPOINT, json={"user": "linkedin", "password": "linkedin"})
    return provider_bindings_token

def get_organizations():
    token = authenticate_against_p19k()
    organizations = requests.get(format(ORGANIZATIONS_ENDPOINT, LINKEDIN_ORGID),
                                 headers={"x-cwi-binding-token": token.access_token})
    # TODO: Load into dataframe
    
def get_accounts():
    token = authenticate_against_p19k()
    accounts = requests.get(format(ACCOUNTS_ENDPOINT, LINKEDIN_ORGID, COURSE_ID),
                                 headers={"x-cwi-binding-token": token.access_token})
    # TODO: Load into dataframe

def get_transactions():
    token = authenticate_against_p19k()
    accounts = requests.get(format(TRANSACTIONS_ENDPOINT, LINKEDIN_ORGID, COURSE_ID, ),
                                 headers={"x-cwi-binding-token": token.access_token})

# function to import data from p19k linkedin binding
def import_p19k_data() -> pd.DataFrame:
    

# TODO: Wrap the json into ndjson -> dataframe
# def sample_p19k_request():
#     r = requests.get('http://localhost:3000/health')
#     print(r.status_code)
#     print(r.content)

In [ ]:
from cwi_notebook import data_loader
# tags: evaluate
# function to find most popular courses, returns sorted list of courses by popularity
# todo: add date filter to this like "entries after X"
def evaluate(data: pd.DataFrame) -> pd.DataFrame:
    df = data[['entry_id', 'line_number', 'account_id', 'amount', 'posting_date']]
    # todo: assumption here is that entry_id is unique
    df = df.drop_duplicates('entry_id')
    df = df.sort_values(['amount'])
    return df

# replace sample loader with p19k_data call above
print(evaluate(data_loader.load_data(inputs)))

In [ ]:
# tags: visualization
# here the chart and date selector will go